Finite-Volume Method

Most popular method in computational fluid dynamics (everyone loves it)

* FVM discretizations are very general and are very flexible

* FVM gives a conservative discretization automatically by directly using the conservation laws in integral form.

Conservation:
$$\frac{\partial}{\partial t} \int_{\text{cv}} e dV + \oint_{\text{cs}} \vec{F} \cdot d\vec{A} = 0$$

Why not make the control volume itself our computational cell?

$$e_i = \frac{1}{\Delta x} int_{x_i - \Delta x/2}^{x_i + \Delta x/2} e(x,t) dx$$

If we know the flux terms at the boundaries, the general conservation law gives:

$$\frac{\partial}{\partial t} e_i + \frac{1}{\Delta x} \left[ F(x_{i + 1/2},t) - F(x_{i-1/2},t)\right] = 0$$

Godunov's Method (1959)

Uses integral form of the conservation laws and a piecewise constant representation of the solution

Use Riemann solution at each cell boundary (what a smart guy)

Need to solve Riemann from $t$ to $\Delta t$ one on each cell boundary

$$F_{i+1/2} = \frac{1}{\Delta t} \int_{t^n}^{t^{n+1}} F(e(x_{i+1/2},t)) dt$$

Rusanov flux aka Lax-Friedrichs flux

$$F_{i+1/2} = \frac{1}{2} [F(e_L) + F(e_R)] - \frac{1}{2} max |F'(e)| (e_R - e_L)$$

F'(e) is the Jacobian of the flux function and max|F'(e)| is the local propagation speed of the fastest traveling wave.

The Riemann solutions at each cell boundary don't interact if $max|F'(e)| \leq \frac{\Delta x}{\Delta t}$ which leads to:
$$F_{i+1/2} = \frac{1}{2} \left( F(e_i) + F(e_{i+1}) - \frac{\Delta x}{\Delta t} (e_{i+1} - e_i) \right)$$

In [3]:
from traffic import rho_red_light, computeF